## Re-Working Clavis 

In [1]:
## This is Clavis 
## Clavis is a Keyword Expansion and Categorization Tool for Digital Marketing & URL Generation 

## For example if you have a keyword like "best shoes" and you want to expand it to "best shoes for cycling" and "best shoes for running"
## as well as categorize whether it is "Male", "Female", "Unisex" or "Kids" you can do that with Clavis
## Also identify whether a keyword is Generic or Branded (Generic = "best shoes", Branded = "Nike shoes")
## also it identifies the Topic of the keyword ("Accessories", "Cycling Jackets", "Mountain Biking Jackets", etc..)
## lastly it will also identify which ones should be "Deleted" and which ones should be "Kept"

## The input to this system are two different CSV Files: 

## Input 1: Keywords.csv
## Keywords File: This is a CSV file with the following columns: Keywords 
## Keywords File Example:
## Keywords: Best shoes, running shoes, etc.. 

## Input 2: Categories.csv
## Categories File: This is a CSV with the following columns: Category, Search For, Return 
## Categories File Example:
## Category | Search For | Return
## Gender | | Unisex --> If Search For is empty, return Unisex 
## Gender | man | Men 
## Gender | woman | Women 
## Gender | lady | Women
## Gender | boy | Kids
## Branded_generic | | Generic --> If Search For is empty, return Generic
## Branded_generic | nike | Branded
## Branded_generic | adidas | Branded
## Branded_generic | puma | Branded
## Branded_generic | reebok | Branded
## Topic | | None --> If Search For is empty, return None
## Topic | cleats | accessories
## topic | vest | Gilet 
## Topic | jacket*cycling*mountain*jacket | mountain biking jackets --> asterisk is a wildcard character, it can be used to match multiple words

## Output:
## Output is a CSV file with the following columns: Keywords, Gender, Branded_Generic, Topic, Delete_Keep
## Output Example:
## Keywords | Gender | Branded_Generic | Topic | Delete_Keep
## Best shoes | Unisex | Generic | None | Keep
## Best shoes for cycling | Unisex | Generic | cycling | Keep
## Best shoes for running | Unisex | Generic | running | Keep
## Nike shoes | Unisex | Branded | None | Keep
## Nike shoes for cycling | Unisex | Branded | cycling | Keep
## Nike shoes for running | Unisex | Branded | running | Keep
## Adidas shoes | Unisex | Branded | None | Keep
## women's cycling suit | Women | Generic | cycling | Keep



## PART 3: URL Generation 
"""

Product - URL	Model - URL	Style - URL	Gender - URL	Sport - URL	Best for - URL	Colour - URL	Features - URL	Collection - URL	Brand - URL	Size - URL	Rise - URL	Sustainable - URL	Material - URL	Teams - URL	Kit Teams - URL	Winter - URL	Outlet - URL	Support - URL	Length - URL	Fit - URL	Surface - URL	Techologies - URL
"""
## those are all the mappings that are available for the URL Generation
## the matching will be done for each of the categories and then concatenated with a custom separator (default is "-")

## There are two parts to this: 
## Part 1. Keyword Search Volume & Expansion This is done with the custom scripts we have under KeywordSearchVolume module [DONE]
## Part 2. Keyword Categorization This is done in the aforementioned fashion from the two CSV Files [DONE]
## Part 3. URL Generation - This is done with an extra mapping file that maps the keywords to the URLs 

In [27]:
!pipenv install matplotlib openpyxl

Installing matplotlib...
Adding matplotlib to Pipfile's [packages]...
✔ Installation Succeeded 
Installing openpyxl...
Adding openpyxl to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock (ee2ed9) out of date, updating to (603c06)...
Locking [dev-packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (603c06)!
Installing dependencies from Pipfile.lock (603c06)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 0/0 — 00:00:00


In [2]:
## importing the libraries 
import streamlit as st 
import pandas as pd 
import numpy as np 
from KeywordSearchVolume.search_volume_extractor import run_search_volume

In [3]:
## Part 0. Upload files
catz = pd.read_csv("/Users/eric/Documents/Locaria/Projects/Clavis/sample_data/categories.csv", sep=';')
kwds = pd.read_csv("/Users/eric/Documents/Locaria/Projects/Clavis/sample_data/keywords.csv")

## Part 1. Keyword Search Volume
keyword_list = kwds['keywords'].tolist()
language = 'English' ## this has to be a streamlit input 
ideas = True ## this has to be a streamlit radio button
geo = 'United States' ## this has to be a streamlit input
payload = dict(keywords=keyword_list, language=language, geo=geo, ideas=ideas)

## get the results from the search volume extractor
results = run_search_volume(**payload)

## parse the results as it is a dictionary, and we need to label those keywords which are "ideas" and were not in the original list
## we will label them as "idea" and "not idea" respectively
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['search_volume']).reset_index().rename(columns={'index':'keywords'})

## add a column to the dataframe to label the keywords as "idea" or "not idea"
results_df['idea'] = np.where(results_df['keywords'].isin(keyword_list), 'Expanded', 'Original')

## expansion factor - is the number of keywords that were generated by the search volume extractor
## divided by the number of keywords that were in the original list
expansion_factor = len(results_df)/len(keyword_list)
print(f"The expansion factor is {expansion_factor:.2f}x")

The expansion factor is 16.44x


In [25]:
results_df

,keywords,search_volume,idea
0,road cleats,70.0,Expanded
1,winter cycling shorts,10.0,Expanded
2,road pedals and cleats,10.0,Expanded
3,cycling bib shorts,1600.0,Expanded
4,cycling shorts,8100.0,Expanded
...,...,...,...
19957,bike puncture repair kit and pump,10.0,Original
19958,co2 bike pump kit,10.0,Original
19959,chariot bike trailer kit,10.0,Original
19960,bike hut cleaning kit,NaN,Original


In [15]:
## Part 2. Keyword Categorization
## we will use the categories.csv file to categorize the keywords

## first we will create a function to categorize the keywords
def categorize_keywords(keywords_df: pd.DataFrame, categories_df: pd.DataFrame) -> pd.DataFrame:
    """Function to categorize the keywords given the categories file"""
    ## copy the df just in case 
    df = keywords_df.copy()
    ## we will loop through the categories file and apply the rules to the keywords
    for index, row in categories_df.iterrows():
        ## if the search for column is empty, we will apply the return column to the keywords
        if pd.isna(row['Search For']):
            df[row['Category']] = row['Return']
        ## if the search for column is not empty, we will apply the return column to the keywords
        ## that contain the search for column
        else:
            try:
                df[row['Category']] = np.where(df['keywords'].str.contains(row['Search For'], case=False), row['Return'], df[row['Category']])
            ## except a KeyError if the category column does not exist in the dataframe
            except KeyError:
                df[row['Category']] = np.where(df['keywords'].str.contains(row['Search For'], case=False), row['Return'], 0)
    return df

In [153]:
## the categories file that will be used as input will be an excel file, with a single sheet but multiple columns
## where the first row will be the column names (i.e. the Category), for example A1, B1, C1, D1, etc.. 
## *Note the Category name can span multiple columns, for example A1, A2, A3, A4, etc.. so we need to find the distance 
## then the second row will contain Search For and Return, which is the same spanning as the Category 
## then the third row will contain the actual values for the Search For and Return, for example A3, B3, C3, D3, etc..
## this will have to be parsed and converted into the format that the function above expects

## load the excel file 
excel_file = "/Users/eric/Documents/Locaria/Projects/Clavis/sample_data/Keyword-Categorization-Mapping-Config.xlsx"
sheet_name = "Config - Categorisation"
categories_df = pd.read_excel(excel_file, sheet_name=sheet_name)
## now we need to clean it up to make it usable for the function above
## extract the category names, exclude Unnamed: 1, 2, etcc 
categories = [x for x in categories_df.columns if 'Unnamed' not in x]
to_drop = [x for x in categories_df.columns[2::3]]
## drop them 
categories_df = categories_df.drop(to_drop, axis = 1)


,Gender,Unnamed: 1,Branded / Generic,Unnamed: 4,Topic,Unnamed: 7,Word Intent,Unnamed: 10,Delete,Unnamed: 13,...,Support - URL,Unnamed: 70,Length - URL,Unnamed: 73,Fit - URL,Unnamed: 76,Surface - URL,Unnamed: 79,Techologies - URL,Unnamed: 82
0,Search For,Return,Search For,Return,Search For,Return,Search For,Return,Search For,Return,...,Search For,Return,Search For,Return,Search For,Return,Search For,Return,Search For,Return
1,NaN,Unisex,NaN,Generic,ciclis,cycling main URL,qué,Long Tail,aa platinum,Delete,...,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1
2,hombre,Men,adidas,Branded,bici,cycling main URL,como,Long Tail,decathlon,Delete,...,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2
3,mujer,Women,originals,Branded,material,cycling main URL,cómo,Long Tail,intersport,Delete,...,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3
4,senora,Women,sportswear,Branded,equipamiento,cycling main URL,adecuado,Long Tail,barato,Delete,...,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,world of warcraft,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wwe,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yulu,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zuru,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# ## rename the "Unnamed X" columns to the previous column name 
# cols_to_rename = categories_df.columns[1::2]
# rename_to = categories_df.columns[::2]
# assert len(cols_to_rename) == len(rename_to)
# rename_dict = dict(zip(cols_to_rename, rename_to))
# categories_df = categories_df.rename(columns=rename_dict)
# ## now we combine the first row with the column names 
# col_names = categories_df.columns.to_list()
# ## searchFor & Return 
# search_for_return = categories_df.iloc[0].values
# ## concatenate the two 
# col_names = [x + '__' + y for x, y in zip(col_names, search_for_return)]
# ## rename the columns
# categories_df.columns = col_names
# ## drop the first row
# categories_df = categories_df.drop(0, axis=0)
# ## now we need to convert the dataframe to the format that the function above expects
# ## we will create a new dataframe with the columns: Category, Search For, Return
# new_df = pd.DataFrame(columns=['Category', 'Search For', 'Return'])
## loop through the columns and extract the values
# for col in categories_df.columns:
#     ## extract the category name
#     category = col.split('__')[0]
#     ## extract the search for and return values
#     search_for = categories_df[col].iloc[0]
#     return_ = categories_df[col].iloc[1]
#     ## append to the new dataframe
#     new_df = new_df.append({'Category': category, 'Search For': search_for, 'Return': return_}, ignore_index=True)
# ## now we can use the function above to categorize the keywords

In [151]:
categories_df

,Gender,Unnamed: 1,Branded / Generic,Unnamed: 4,Topic,Unnamed: 7,Word Intent,Unnamed: 10,Delete,Unnamed: 13,...,Support - URL,Unnamed: 70,Length - URL,Unnamed: 73,Fit - URL,Unnamed: 76,Surface - URL,Unnamed: 79,Techologies - URL,Unnamed: 82
0,Search For,Return,Search For,Return,Search For,Return,Search For,Return,Search For,Return,...,Search For,Return,Search For,Return,Search For,Return,Search For,Return,Search For,Return
1,NaN,Unisex,NaN,Generic,ciclis,cycling main URL,qué,Long Tail,aa platinum,Delete,...,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1
2,hombre,Men,adidas,Branded,bici,cycling main URL,como,Long Tail,decathlon,Delete,...,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2
3,mujer,Women,originals,Branded,material,cycling main URL,cómo,Long Tail,intersport,Delete,...,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3
4,senora,Women,sportswear,Branded,equipamiento,cycling main URL,adecuado,Long Tail,barato,Delete,...,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,world of warcraft,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wwe,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yulu,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zuru,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Gender,Gender,Branded / Generic,Branded / Generic,Topic,Topic,Word Intent,Word Intent,Delete,Delete,...,Support - URL,Support - URL,Length - URL,Length - URL,Fit - URL,Fit - URL,Surface - URL,Surface - URL,Techologies - URL,Techologies - URL
0,Search For,Return,Search For,Return,Search For,Return,Search For,Return,Search For,Return,...,Search For,Return,Search For,Return,Search For,Return,Search For,Return,Search For,Return
1,NaN,Unisex,NaN,Generic,ciclis,cycling main URL,qué,Long Tail,aa platinum,Delete,...,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1,Category 1
2,hombre,Men,adidas,Branded,bici,cycling main URL,como,Long Tail,decathlon,Delete,...,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2,Category 2
3,mujer,Women,originals,Branded,material,cycling main URL,cómo,Long Tail,intersport,Delete,...,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3,Category 3
4,senora,Women,sportswear,Branded,equipamiento,cycling main URL,adecuado,Long Tail,barato,Delete,...,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4,Category 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,world of warcraft,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wwe,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yulu,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zuru,Delete,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
catz[catz['Category'] == 'Delete']

,Category,Search For,Return
212,Delete,m:part,Delete
213,Delete,macgregor,Delete
214,Delete,macron,Delete
215,Delete,madison,Delete
216,Delete,magic bodyfashion,Delete
...,...,...,...
841,Delete,tom smith,Delete
842,Delete,tommy sport,Delete
843,Delete,tomy,Delete
844,Delete,unbranded,Delete


In [17]:
catz

,Category,Search For,Return
0,Gender,NaN,Unisex
1,Gender,man,Men
2,Gender,woman,Women
3,Gender,lady,Women
4,Gender,woman,Women
...,...,...,...
841,Delete,tom smith,Delete
842,Delete,tommy sport,Delete
843,Delete,tomy,Delete
844,Delete,unbranded,Delete


In [18]:
results_df

,keywords,search_volume,idea
0,road cleats,70.0,Expanded
1,winter cycling shorts,10.0,Expanded
2,road pedals and cleats,10.0,Expanded
3,cycling bib shorts,1600.0,Expanded
4,cycling shorts,8100.0,Expanded
...,...,...,...
19957,bike puncture repair kit and pump,10.0,Original
19958,co2 bike pump kit,10.0,Original
19959,chariot bike trailer kit,10.0,Original
19960,bike hut cleaning kit,NaN,Original


In [19]:
res = categorize_keywords(results_df, catz)

In [20]:
res

,keywords,search_volume,idea,Gender,Branded_generic,Topic,Delete
0,road cleats,70.0,Expanded,Unisex,Generic,accessories,0
1,winter cycling shorts,10.0,Expanded,Unisex,Generic,0,0
2,road pedals and cleats,10.0,Expanded,Unisex,Generic,accessories,0
3,cycling bib shorts,1600.0,Expanded,Unisex,Generic,0,0
4,cycling shorts,8100.0,Expanded,Unisex,Generic,0,0
...,...,...,...,...,...,...,...
19957,bike puncture repair kit and pump,10.0,Original,Unisex,Generic,0,0
19958,co2 bike pump kit,10.0,Original,Unisex,Generic,0,0
19959,chariot bike trailer kit,10.0,Original,Unisex,Generic,0,0
19960,bike hut cleaning kit,NaN,Original,Unisex,Generic,0,0
